<a href="https://colab.research.google.com/github/2kunal6/UniBonn/blob/master/cuda_lab/2/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
import torchvision
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim

import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

np.random.seed(1)

transform_list = transforms.Compose([
            transforms.Grayscale(num_output_channels=1),
            transforms.ToTensor()
            #transforms.Normalize([0.5,],[0.5,])
        ])
train = datasets.CIFAR10("./", train=True, transform=transform_list, download=True)

test = torch.utils.data.DataLoader(
    datasets.CIFAR10("./", train=False, transform=transform_list, download=True), batch_size = 32, shuffle = True
)

print("datasets loaded")

IMG_SIZE = 32*32

'''for data in train:
  X, y = data[0][0], data[1][0]
  plt.imshow(X.view(32, 32))
  print("Label: " + str(y))
  plt.show()
  break'''

class LogisticRegression(nn.Module):
  def __init__(self, input_dim, output_dim):
    super(LogisticRegression, self).__init__()
    self.linear1 = nn.Linear(input_dim, 300)
    self.linear2 = nn.Linear(300, output_dim)
    self.relu = nn.ReLU()

  def forward(self, x):
    out = self.relu(self.linear1(x))
    out = self.linear2(out)
    return out

if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

model = LogisticRegression(IMG_SIZE, 10).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

criterion = nn.CrossEntropyLoss().to(device)

def test_perform(test_data):
  total = 0
  correct = 0

  y_true = []
  y_pred = []
  with torch.no_grad():
    for data in test_data:
      X, y = data[0].to(device), data[1].to(device)

      output = model(X.view(-1, IMG_SIZE))
      _, predicted = torch.max(output, 1)

      total += y.size(0)
      correct += (predicted == y).sum().item()

      y_pred.extend(predicted.tolist())
      y_true.extend(y.tolist())

    accuracy = 100 * correct / total
    return accuracy, y_true, y_pred


for epoch in range(20):
  train_size = int(0.9 * len(train))
  val_size = len(train) - train_size
  train_dataset, val_dataset = torch.utils.data.random_split(train, [train_size, val_size])
  train_loader = torch.utils.data.DataLoader(train_dataset,batch_size = 32, shuffle = True)
  for data in train_loader:
    X, y = data[0].to(device), data[1].to(device)
    optimizer.zero_grad()
    output = model(X.view(-1, IMG_SIZE))
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()
  print(f'epoch: {epoch}, loss: {loss}')

    
accuracy, y_true, y_pred = test_perform(test)

print(confusion_matrix(y_true, y_pred))
print(f'Accuracy of the network on the 10000 test images: {accuracy}')

Files already downloaded and verified
Files already downloaded and verified
datasets loaded
Running on the GPU
epoch: 0, loss: 2.2840306758880615
epoch: 1, loss: 2.0513477325439453
epoch: 2, loss: 2.186150550842285
epoch: 3, loss: 2.3392558097839355
epoch: 4, loss: 2.489842414855957
epoch: 5, loss: 2.0354650020599365
epoch: 6, loss: 1.5546187162399292
epoch: 7, loss: 1.9133005142211914
epoch: 8, loss: 2.123992681503296
epoch: 9, loss: 2.41884446144104
epoch: 10, loss: 1.9515855312347412
epoch: 11, loss: 2.1631078720092773
epoch: 12, loss: 2.29356050491333
epoch: 13, loss: 1.825661063194275
epoch: 14, loss: 2.166281223297119
epoch: 15, loss: 2.1993050575256348
epoch: 16, loss: 1.7991697788238525
epoch: 17, loss: 2.2662792205810547
epoch: 18, loss: 2.0974225997924805
epoch: 19, loss: 1.7113345861434937
[[169 203   0  27 543   3   0   0   3  52]
 [120 371   0  31 173   3   0   0   2 300]
 [171  94   0  50 654  10   0   0   1  20]
 [144 145   0 167 457  55   0   1   6  25]
 [154 114   0  4